In [68]:
# An MNIST loader.

import numpy as np
import gzip
import struct


def load_images(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Read the header information into a bunch of variables:
        _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
        # Read all the pixels into a NumPy array of bytes:
        all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
        # Reshape the pixels into a matrix where each line is an image:
        return all_pixels.reshape(n_images, columns * rows)


def prepend_bias(X):
    # Insert a column of 1s in the position 0 of X.
    # (“axis=1” stands for: “insert a column, not a row”)
    return np.insert(X, 0, 1, axis=1)


# 60000 images, each 785 elements (1 bias + 28 * 28 pixels)
X_train = prepend_bias(load_images("train-images-idx3-ubyte.gz"))

# 10000 images, each 785 elements, with the same structure as X_train
X_test = prepend_bias(load_images("t10k-images-idx3-ubyte.gz"))


def load_labels(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Skip the header bytes:
        f.read(8)
        # Read all the labels into a list:
        all_labels = f.read()
        # Reshape the list of labels into a one-column matrix:
        return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)


def one_hot_encode(Y):
    n_labels = Y.shape[0]
    n_classes = 10
    encoded_Y = np.zeros((n_labels, n_classes))
    for i in range(n_labels):
        label = Y[i]
        encoded_Y[i][label] = 1
    return encoded_Y


# 60K labels, each a single digit from 0 to 9
Y_train_unencoded = load_labels("train-labels-idx1-ubyte.gz")

# 60K labels, each consisting of 10 one-hot encoded elements
Y_train = one_hot_encode(Y_train_unencoded)

# 10000 labels, each a single digit from 0 to 9
Y_test = load_labels("t10k-labels-idx1-ubyte.gz")


In [69]:
def sigmoid(x):
  return np.exp(x)/(1+np.exp(x))

def loss(w):
  Yhat = sigmoid(np.matmul(X,w))
  return -np.sum(Y*np.log(Yhat)+(1-Y)*np.log(1-Yhat))/X.shape[0]

def dloss(w):
  Yhat = sigmoid(np.matmul(X,w))
  #return -np.mean((Y-Yhat)*X,axis=0).reshape([-1,1])
  return -np.matmul(X.T,(Y-Yhat))/X.shape[0]

def train(lr,epochs):
  w = np.zeros((785,10))
  for i in range(0,epochs):
    w = w - lr*dloss(w)
    print(loss(w))
  return w

w = train(.00001,10)

8.434456875083336
5.512047488923876
2.956870073593654
1.8985387657057096
1.7558289155266744
1.674881272926218
1.623875243420281
1.5652805689746652
1.5292692651055577
1.4834968500183898


In [79]:
Yhat=sigmoid(np.matmul(X_test,w))

In [80]:
predictions=np.argmax(Yhat,axis=1).reshape([-1,1])

In [81]:
Y_test-predictions

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [ 0],
       [-3],
       [ 0]])

In [82]:
misClassifications=np.count_nonzero(Y_test-predictions)

In [83]:
10000-misClassifications

8355